# Часть 1 - Манипуляции с E-Utilities

## 0. Импорт библиотеки

## 1. Запросим информацию по последовательностям, содержащим выбранный нами ген

In [1]:
from Bio import Entrez
from Bio import SeqIO
import xml.etree.ElementTree as ET

## 2. Построим сводную таблицу

In [2]:
Entrez.email = "stephenpetrosian@gmail.com"
handle = Entrez.esearch(db="nucleotide", term='"Human"[Organism] AND GSPT1[Gene]')
record_seq = Entrez.read(handle)

In [8]:
for id in record_seq['IdList']:
    handle = Entrez.esummary(db="nucleotide", id=id)
    record = Entrez.read(handle)
    print("Protein info\nUID: {}\nAccession number: {}\nSequence length: {}\n".
          format(record[0]["Id"], record[0]["Caption"],record[0]["Length"]))

Protein info
UID: 1676355513
Accession number: NM_001130006
Sequence length: 7138

Protein info
UID: 1676319656
Accession number: NM_001130007
Sequence length: 7166

Protein info
UID: 1519312966
Accession number: NM_002094
Sequence length: 7141

Protein info
UID: 568815582
Accession number: NC_000016
Sequence length: 90338345

Protein info
UID: 74273666
Accession number: CM000267
Sequence length: 75226909

Protein info
UID: 74230029
Accession number: CH471112
Sequence length: 14690834

Protein info
UID: 33874733
Accession number: BC009503
Sequence length: 2523

Protein info
UID: 39754980
Accession number: AY398991
Sequence length: 1562

Protein info
UID: 307685420
Accession number: AB590486
Sequence length: 1919



## 3. Скачивание запрошенных нами последовательностей

In [3]:
for id in record_seq['IdList']:
    handle = Entrez.efetch(db="nucleotide", id=id, rettype="fasta", retmode="text")
    with open('entrez.fasta', 'a') as fasta_file:
        fasta_file.write(handle.read())

## 4. Скачивание всех последовательностей из заданной статьи

In [4]:
handle_PMID = Entrez.efetch(db="pubmed", id=12890024, retmode="xml")
root = ET.fromstring(handle_PMID.read())
for neighbor in root.iter('AccessionNumber'):
    handle = Entrez.efetch(db="nucleotide", id=neighbor.text, rettype="fasta", retmode="text")
    with open('entrezPMID.fasta', 'a') as fasta_file:
        fasta_file.write(handle.read())

# Часть 2 - Множественное выравнивание

## 1. Описание данных

|  Организм            	| Белок                                                       	| Ген   	|
|----------------------	|-------------------------------------------------------------	|-------	|
| Kluyveromyces lactis 	| Eukaryotic peptide chain release factor GTP-binding subunit 	| SUP35 	|

## 2. Сравнение алгоритмов выравнивания

Запустим ряд алгоритмов выравнивания и сравним их время работы.

|      | Muscle   | Mafft    | Prank    | Kalign   |
|------|----------|----------|----------|----------|
| Time | 0m5,220s | 0m3,613s | 0m9,176s | 0m0,396s |

## 3. Что не так с SUP35_10seqs_strange_aln.fa?

Пропуски. Их много. Чтобы исправить это, лучше всего сравнить деревья, полученные с использованием всех данных или только отфильтрованных позиций, и в случае серьёзных различий решать, какое выравнивание лучше отражает биологическую ситуацию.

## 4. Получение белка и его выравнивание

Получим аминокислотную последовательность

In [6]:
with open("SUP35_10seqs.fa", "r") as handle, open('SUP35_10_prot.fa', 'a') as prot_file:
    for record in SeqIO.parse(handle, "fasta"):
        prot_file.write('>')
        prot_file.write(record.id)
        prot_file.write('\n')
        prot_file.write(str(record.seq.translate()))
        prot_file.write('\n')

Запустим на ней ряд алгоритмов выравнивания и сравним время работы.

|      | Muscle   | Mafft    | Prank    | Kalign   |
|------|----------|----------|----------|----------|
| Time | 0m0,281s | 0m0,566s |  0m14,131s | 0m0,082s |